In [1]:
# Automatically reload changes to external code
%load_ext autoreload
%autoreload 2



In [2]:
import tensorflow as tf
from mnist import loader #loader for mnist dataset
from matplotlib import pyplot as plt
from tensorflow.python.ops import rnn
import numpy as np
import pdb, time

# Mnist dataset

The MNIST database of handwritten digits. [[website]](http://yann.lecun.com/exdb/mnist/)<br>
There are **60,000** training images and **10,000** testing images in this dataset.<br>
Each digit is a one-channel image. Size of image = 28*28 = 784.

![](imgs/mnist_ex.png)

There are some build-in mnist function can be used in tensorflow.

Ex.<br>
from tensorflow.examples.tutorials.mnist import input_data<br>
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instead of using these functions, I'll use the orginal dataset manually in this code.<br>
It's more clear to trace the data-processing.

When we load the data


In [3]:
# load mnist data manually
# loading 'train' or 'test' data
# ex. load_mnist_data('train')
# return images, labels and mean of all images. (But, we'll only use the mean of training data.)
# ims: [N * 784]
# labels: [N]
# ims_mean: [784]

def load_mnist_data(flag, data_path='data'):
    data_loader = loader.MNIST(data_path)
    if flag == 'train':
        ims, labels = data_loader.load_training()
    elif flag == 'test':
        ims, labels = data_loader.load_testing()
    else:
        raise ValueError("Error. Only training or testing data.")
    ims = ims/255.0
    ims_mean = np.mean(ims, axis=0)
    ims = np.reshape(ims, [len(ims),28,28])
    ims_mean = np.reshape(ims_mean,(28,28))
    return ims, labels, ims_mean

In [4]:
# Parameters
learning_rate = 0.01
training_epochs = 1
batch_size = 100   # training batch size
test_batch_size = 100
display_step = 50  # testing 

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)
stddev=0.01    # standard deviation for random initialization

# Functions of Recurrent Neural Network

In [5]:
def RNN():
    rnn_cell = tf.nn.rnn_cell
    # tf Graph input
    x = tf.placeholder("float", [None, n_steps, n_input])  # mnist input images, [batch_size x 28 x 28]
    y = tf.placeholder(tf.int32,[None])              # label, [batch_size]
    # Declare Variables 
    # Store layers weight & bias
    weights = {
        'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
    }
    biases = {
        'out': tf.Variable(tf.random_normal([n_classes]))
    }

    ###### Define a LSTM cell with tensorflow #######
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden)
    # Initial zero state
    state = lstm_cell.zero_state(batch_size, tf.float32)
    for i in range(n_steps):
        # Get RNN cell output
        with tf.variable_scope('lstm', reuse=True if i > 0 else None):
            output, state = lstm_cell(x[:,i,:], state)
    
    pred = tf.matmul(output, weights['out']) + biases['out']

    probs = tf.nn.softmax(pred)
    
    one_hot_y = tf.one_hot(y, n_classes, on_value=1, off_value=0, axis=-1)
    cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=pred)
        
    loss = tf.reduce_mean(cross_entropy_loss)
    
    return x, y, loss, pred, one_hot_y


In [6]:
def eval_model(sess, x, y, ims, labels, ims_test, labels_test, ims_mean, iter_per_epoch, test_iter):
    Train_Loss = 0
    Test_Loss = 0
    Train_Acc = 0
    Test_Acc = 0
    for idx in xrange(iter_per_epoch):
        batch_xs = ims[order_list[idx*batch_size:(idx+1)*batch_size]] - ims_mean
        batch_ys = labels[order_list[idx*batch_size:(idx+1)*batch_size]]
        C, A = sess.run([cost, accuracy], feed_dict={x: batch_xs, y: batch_ys})
        Train_Loss += C/batch_size   # calculate the loss in average (per image).
        Train_Acc += A
    # Eval testing dataset
    for idx in xrange(test_iter):
        batch_xs = ims_test[order_list[idx*test_batch_size:(idx+1)*test_batch_size]] - ims_mean
        batch_ys = labels_test[order_list[idx*test_batch_size:(idx+1)*test_batch_size]]
        C, A = sess.run([cost, accuracy], feed_dict={x: batch_xs, y: batch_ys})
        Test_Loss += C/test_batch_size
        Test_Acc += A
    return Train_Loss, Train_Acc, Test_Loss, Test_Acc

# Main function



In [7]:
# loading training and testing data
ims, labels, ims_mean = load_mnist_data('train', data_path='MNIST_data')
#ims_mean = np.zeros((28*28))
ims_test, labels_test, _ = load_mnist_data('test', data_path='MNIST_data')

order_list = range(len(ims))

# parameters related to mnist dataset 
test_iter = len(ims_test)/test_batch_size # number of testing-minibatch.

iter_per_epoch = len(ims)/batch_size      # number of training-minibatch.

# Launch the graph
with tf.Session() as sess:

    # Keep training until reach max iterations
    x, y, cost, pred, one_hot_y = RNN()
    train_loss = cost/batch_size # loss per image
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(train_loss)


    # Evaluate model
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(one_hot_y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


    # initialize all variables
    init = tf.global_variables_initializer()
    sess.run(init)
    Loss_plt = {'x':[], 'train_y':[], 'test_y':[]}
    Acc_plt  = {'x':[], 'train_y':[], 'test_y':[]} 
    # Before Training (Random initialization), Evaluate the model one-time.
    begin = time.time()
    Train_Loss, Train_Acc, Test_Loss, Test_Acc=eval_model(sess, x, y, ims, labels, ims_test, labels_test, ims_mean, iter_per_epoch, test_iter)
    print "------After Random Initialization------"
    print "Training: loss=%f, acc=%f.\t\tTesting: loss=%f, acc=%f" %(Train_Loss/iter_per_epoch, Train_Acc/iter_per_epoch,
                                                                     Test_Loss/test_iter, Test_Acc/test_iter)
    
    epoch = 0
    step = 0      
    
    Loss_plt['x'].append(float(step)/iter_per_epoch)
    Loss_plt['train_y'].append(Train_Loss/iter_per_epoch)
    Loss_plt['test_y'].append(Test_Loss/test_iter)
    Acc_plt['x'].append(float(step)/iter_per_epoch)
    Acc_plt['train_y'].append(Train_Acc/iter_per_epoch)
    Acc_plt['test_y'].append(Test_Acc/test_iter)
    
    duration = time.time()-begin
    print " %f seconds"%(duration)
    
    print "------Start Training------"

    for epoch in xrange(training_epochs):
        begin = time.time()
        Train_Loss = 0
        Test_Loss = 0
        Train_Acc = 0
        Test_Acc = 0
        for idx in xrange(iter_per_epoch):
            batch_xs = ims[order_list[idx*batch_size:(idx+1)*batch_size]] - ims_mean
            batch_ys = labels[order_list[idx*batch_size:(idx+1)*batch_size]]
            # Run optimization op (backprop)
            sess.run([optimizer], feed_dict={x: batch_xs, y: batch_ys})
            if step % display_step == 0:
                Train_Loss, Train_Acc, Test_Loss, Test_Acc=eval_model(sess, x, y, ims, labels, ims_test, labels_test, ims_mean, iter_per_epoch, test_iter)
                print "Epoch %f, Training: loss=%f, acc=%f.\t\tTesting: loss=%f, acc=%f"%(float(step)/iter_per_epoch, Train_Loss/iter_per_epoch, Train_Acc/iter_per_epoch, 
                                                                                         Test_Loss/test_iter, Test_Acc/test_iter)
                Loss_plt['x'].append(float(step)/iter_per_epoch)
                Loss_plt['train_y'].append(Train_Loss/iter_per_epoch)
                Loss_plt['test_y'].append(Test_Loss/test_iter)
                Acc_plt['x'].append(float(step)/iter_per_epoch)
                Acc_plt['train_y'].append(Train_Acc/iter_per_epoch)
                Acc_plt['test_y'].append(Test_Acc/test_iter)
            step += 1
        
        # Evaluate after each epoch finished.
        Train_Loss, Train_Acc, Test_Loss, Test_Acc=eval_model(sess, x, y, ims, labels, ims_test, labels_test, ims_mean, iter_per_epoch, test_iter)
        print "Epoch %d, Training: loss=%f, acc=%f.\t\tTesting: loss=%f, acc=%f"%(epoch+1, Train_Loss/iter_per_epoch, Train_Acc/iter_per_epoch, 
                                                                                         Test_Loss/test_iter, Test_Acc/test_iter)
        Loss_plt['x'].append(float(step)/iter_per_epoch)
        Loss_plt['train_y'].append(Train_Loss/iter_per_epoch)
        Loss_plt['test_y'].append(Test_Loss/test_iter)
        Acc_plt['x'].append(float(step)/iter_per_epoch)
        Acc_plt['train_y'].append(Train_Acc/iter_per_epoch)
        Acc_plt['test_y'].append(Test_Acc/test_iter)
        duration = time.time()-begin
        print "Cost %f seconds"%(duration)
    print("Optimization Finished!") 

------After Random Initialization------
Training: loss=0.026590, acc=0.033717.		Testing: loss=0.026559, acc=0.028300
 3.004302 seconds
------Start Training------
Epoch 0.000000, Training: loss=0.021154, acc=0.302267.		Testing: loss=0.021258, acc=0.304200
Epoch 0.083333, Training: loss=0.005462, acc=0.823450.		Testing: loss=0.005236, acc=0.829900
Epoch 0.166667, Training: loss=0.003870, acc=0.875033.		Testing: loss=0.003840, acc=0.872600
Epoch 0.250000, Training: loss=0.003109, acc=0.906583.		Testing: loss=0.002827, acc=0.914000
Epoch 0.333333, Training: loss=0.002103, acc=0.933650.		Testing: loss=0.001958, acc=0.938000
Epoch 0.416667, Training: loss=0.001434, acc=0.956567.		Testing: loss=0.001372, acc=0.960600
Epoch 0.500000, Training: loss=0.001558, acc=0.952783.		Testing: loss=0.001529, acc=0.953700
Epoch 0.583333, Training: loss=0.001274, acc=0.961950.		Testing: loss=0.001297, acc=0.959700
Epoch 0.666667, Training: loss=0.001677, acc=0.949383.		Testing: loss=0.001792, acc=0.946900
E